In [1]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd
from urllib import parse
import pymysql

In [2]:
url = "http://openapi.airport.co.kr/service/rest/FlightScheduleList/getIflightScheduleList?serviceKey=WRvGBkn9UEtw%2BAsg0tYo210Etxvb1QEcAX%2BwfWvOxVGYJkh1CNZ%2FY4QFa0r7j4bhT4NjPu7z1i1ck8ZgsKMt2Q%3D%3D&schDate=20151005&numOfRows=5000"

이부분에서 고친 부분이 있나여ㅛ??

In [3]:
result = parse.urlparse(url)

In [4]:
result

ParseResult(scheme='http', netloc='openapi.airport.co.kr', path='/service/rest/FlightScheduleList/getIflightScheduleList', params='', query='serviceKey=WRvGBkn9UEtw%2BAsg0tYo210Etxvb1QEcAX%2BwfWvOxVGYJkh1CNZ%2FY4QFa0r7j4bhT4NjPu7z1i1ck8ZgsKMt2Q%3D%3D&schDate=20151005&numOfRows=5000', fragment='')

In [5]:
result.geturl()

'http://openapi.airport.co.kr/service/rest/FlightScheduleList/getIflightScheduleList?serviceKey=WRvGBkn9UEtw%2BAsg0tYo210Etxvb1QEcAX%2BwfWvOxVGYJkh1CNZ%2FY4QFa0r7j4bhT4NjPu7z1i1ck8ZgsKMt2Q%3D%3D&schDate=20151005&numOfRows=5000'

In [6]:
result.scheme

'http'

In [7]:
result[0]

'http'

In [8]:
result.hostname

'openapi.airport.co.kr'

In [9]:
result.path

'/service/rest/FlightScheduleList/getIflightScheduleList'

In [10]:
result.query

'serviceKey=WRvGBkn9UEtw%2BAsg0tYo210Etxvb1QEcAX%2BwfWvOxVGYJkh1CNZ%2FY4QFa0r7j4bhT4NjPu7z1i1ck8ZgsKMt2Q%3D%3D&schDate=20151005&numOfRows=5000'

* ParseResult를 사용하여 url주소 구조 완성 

In [11]:
parse_result = parse.ParseResult(scheme  = result.scheme, netloc= result.hostname, path=result.path, 
                 params = result.params, query=result.query, fragment=result.fragment)

In [12]:
parse_result

ParseResult(scheme='http', netloc='openapi.airport.co.kr', path='/service/rest/FlightScheduleList/getIflightScheduleList', params='', query='serviceKey=WRvGBkn9UEtw%2BAsg0tYo210Etxvb1QEcAX%2BwfWvOxVGYJkh1CNZ%2FY4QFa0r7j4bhT4NjPu7z1i1ck8ZgsKMt2Q%3D%3D&schDate=20151005&numOfRows=5000', fragment='')

* urlunparse() 메소드를 사용하여 주소 형태로 전달

In [13]:
parse.urlunparse(parse_result)

'http://openapi.airport.co.kr/service/rest/FlightScheduleList/getIflightScheduleList?serviceKey=WRvGBkn9UEtw%2BAsg0tYo210Etxvb1QEcAX%2BwfWvOxVGYJkh1CNZ%2FY4QFa0r7j4bhT4NjPu7z1i1ck8ZgsKMt2Q%3D%3D&schDate=20151005&numOfRows=5000'

## 주소의 파라미터 수정하기

In [14]:
query_dict = parse.parse_qs(result.query)

In [15]:
query_dict

{'serviceKey': ['WRvGBkn9UEtw+Asg0tYo210Etxvb1QEcAX+wfWvOxVGYJkh1CNZ/Y4QFa0r7j4bhT4NjPu7z1i1ck8ZgsKMt2Q=='],
 'schDate': ['20151005'],
 'numOfRows': ['5000']}

### doseq 옵션
* False가 기본값이며, dict의 value값의 []까지 인코딩을 한다. 
* True로 설정하면 실제 값만 인코딩을 한다

In [16]:
parse.urlencode(query_dict, doseq=True)

'serviceKey=WRvGBkn9UEtw%2BAsg0tYo210Etxvb1QEcAX%2BwfWvOxVGYJkh1CNZ%2FY4QFa0r7j4bhT4NjPu7z1i1ck8ZgsKMt2Q%3D%3D&schDate=20151005&numOfRows=5000'

In [17]:
api_url = parse.ParseResult(scheme  = result.scheme, netloc= result.hostname, path=result.path, 
                 params = result.params, query=parse.urlencode(query_dict, doseq=True), fragment=result.fragment)

In [18]:
url = parse.urlunparse(api_url)

In [19]:
print (url)

http://openapi.airport.co.kr/service/rest/FlightScheduleList/getIflightScheduleList?serviceKey=WRvGBkn9UEtw%2BAsg0tYo210Etxvb1QEcAX%2BwfWvOxVGYJkh1CNZ%2FY4QFa0r7j4bhT4NjPu7z1i1ck8ZgsKMt2Q%3D%3D&schDate=20151005&numOfRows=5000


## xml 파싱

In [20]:
r = requests.get(url)

In [21]:
root = ET.fromstring(r.text)

In [22]:
total = []
items = root.iter(tag='item')
for element in items:
    #print(element.find("accDefRate").text)
    covid_dict = {}
    for x in element:
        covid_dict[x.tag] = x.text
    total.append(covid_dict)

In [23]:
len(total)

454

In [24]:
df = pd.DataFrame(total)

In [25]:
df.head(2)

,airlineEnglish,airlineKorean,airport,airportCode,city,cityCode,flightPurpose,internationalEddate,internationalFri,internationalIoType,internationalMon,internationalNum,internationalSat,internationalStdate,internationalSun,internationalThu,internationalTime,internationalTue,internationalWed
0,EASTAR JET,이스타항공,청주,CJJ,상해/푸동,PVG,여객기,2015-10-24T00:00:00+09:00,N,IN,N,ZE822,N,2015-07-21T00:00:00+09:00,Y,Y,0230,N,N
1,JEJU AIR,제주항공,대구,TAE,북경/베이징,PEK,여객기,2015-10-24T00:00:00+09:00,N,IN,N,7C8102,Y,2015-03-29T00:00:00+09:00,N,Y,0510,Y,N


In [26]:
df = df.where(~(df == "null"), None)

In [27]:
df.head(2)

,airlineEnglish,airlineKorean,airport,airportCode,city,cityCode,flightPurpose,internationalEddate,internationalFri,internationalIoType,internationalMon,internationalNum,internationalSat,internationalStdate,internationalSun,internationalThu,internationalTime,internationalTue,internationalWed
0,EASTAR JET,이스타항공,청주,CJJ,상해/푸동,PVG,여객기,2015-10-24T00:00:00+09:00,N,IN,N,ZE822,N,2015-07-21T00:00:00+09:00,Y,Y,0230,N,N
1,JEJU AIR,제주항공,대구,TAE,북경/베이징,PEK,여객기,2015-10-24T00:00:00+09:00,N,IN,N,7C8102,Y,2015-03-29T00:00:00+09:00,N,Y,0510,Y,N


In [28]:
df = df.where(pd.notnull(df), None)

In [29]:
from datetime import date, datetime

In [30]:
datetime.strptime('2015-07-21T00:00:00', '%Y-%m-%dT%H:%M:%S')

datetime.datetime(2015, 7, 21, 0, 0)

In [31]:
df['internationalStdate']

0      2015-07-21T00:00:00+09:00
1      2015-03-29T00:00:00+09:00
2      2015-08-16T00:00:00+09:00
3      2015-10-02T00:00:00+09:00
4      2015-09-25T00:00:00+09:00
                 ...            
449    2015-08-16T00:00:00+09:00
450    2015-03-29T00:00:00+09:00
451    2015-10-03T00:00:00+09:00
452    2015-08-16T00:00:00+09:00
453    2015-08-16T00:00:00+09:00
Name: internationalStdate, Length: 454, dtype: object

In [32]:
df['internationalStdate'] = df['internationalStdate'].apply(lambda x: x[:10])

In [33]:
df['internationalEddate'] = df['internationalEddate'].apply(lambda x: x[:10])

In [34]:
df.head(3)

,airlineEnglish,airlineKorean,airport,airportCode,city,cityCode,flightPurpose,internationalEddate,internationalFri,internationalIoType,internationalMon,internationalNum,internationalSat,internationalStdate,internationalSun,internationalThu,internationalTime,internationalTue,internationalWed
0,EASTAR JET,이스타항공,청주,CJJ,상해/푸동,PVG,여객기,2015-10-24,N,IN,N,ZE822,N,2015-07-21,Y,Y,0230,N,N
1,JEJU AIR,제주항공,대구,TAE,북경/베이징,PEK,여객기,2015-10-24,N,IN,N,7C8102,Y,2015-03-29,N,Y,0510,Y,N
2,ASIANA AIRLINE,아시아나항공,청주,CJJ,북경/베이징,PEK,여객기,2015-10-24,Y,IN,N,OZ3745,N,2015-08-16,Y,N,0525,N,Y


In [35]:
df.keys()

Index(['airlineEnglish', 'airlineKorean', 'airport', 'airportCode', 'city',
       'cityCode', 'flightPurpose', 'internationalEddate', 'internationalFri',
       'internationalIoType', 'internationalMon', 'internationalNum',
       'internationalSat', 'internationalStdate', 'internationalSun',
       'internationalThu', 'internationalTime', 'internationalTue',
       'internationalWed'],
      dtype='object')

# 2019, 2020년 데이터 모으기 

In [36]:
query_dict

{'serviceKey': ['WRvGBkn9UEtw+Asg0tYo210Etxvb1QEcAX+wfWvOxVGYJkh1CNZ/Y4QFa0r7j4bhT4NjPu7z1i1ck8ZgsKMt2Q=='],
 'schDate': ['20151005'],
 'numOfRows': ['5000']}

In [37]:
total = []

In [38]:
for x in pd.date_range('2019-01-01', '2019-12-31'):
    print(x)
    query_dict['schDate']  = x.strftime("%Y-%m-%d").replace("-", "")
    api_url = parse.ParseResult(scheme  = result.scheme, netloc= result.hostname, path=result.path, 
                 params = result.params, query=parse.urlencode(query_dict, doseq=True), fragment=result.fragment)
    url = parse.urlunparse(api_url)
    r = requests.get(url)
    root = ET.fromstring(r.text)

    items = root.iter(tag='item')
    for element in items:
        #print(element.find("accDefRate").text)
        covid_dict = {}
        for x in element:
            covid_dict[x.tag] = x.text
        total.append(covid_dict)

2019-01-01 00:00:00
2019-01-02 00:00:00
2019-01-03 00:00:00
2019-01-04 00:00:00
2019-01-05 00:00:00
2019-01-06 00:00:00
2019-01-07 00:00:00
2019-01-08 00:00:00
2019-01-09 00:00:00
2019-01-10 00:00:00
2019-01-11 00:00:00
2019-01-12 00:00:00
2019-01-13 00:00:00
2019-01-14 00:00:00
2019-01-15 00:00:00
2019-01-16 00:00:00
2019-01-17 00:00:00
2019-01-18 00:00:00
2019-01-19 00:00:00
2019-01-20 00:00:00
2019-01-21 00:00:00
2019-01-22 00:00:00
2019-01-23 00:00:00
2019-01-24 00:00:00
2019-01-25 00:00:00
2019-01-26 00:00:00
2019-01-27 00:00:00
2019-01-28 00:00:00
2019-01-29 00:00:00
2019-01-30 00:00:00
2019-01-31 00:00:00
2019-02-01 00:00:00
2019-02-02 00:00:00
2019-02-03 00:00:00
2019-02-04 00:00:00
2019-02-05 00:00:00
2019-02-06 00:00:00
2019-02-07 00:00:00
2019-02-08 00:00:00
2019-02-09 00:00:00
2019-02-10 00:00:00
2019-02-11 00:00:00
2019-02-12 00:00:00
2019-02-13 00:00:00
2019-02-14 00:00:00
2019-02-15 00:00:00
2019-02-16 00:00:00
2019-02-17 00:00:00
2019-02-18 00:00:00
2019-02-19 00:00:00


In [39]:
df = pd.DataFrame(total)

In [40]:
df.shape

(331408, 19)

In [41]:
df2 = df.drop_duplicates().copy()

In [42]:
df2.to_csv("./airport_schedule.csv", index=False, encoding='utf-8-sig')

In [43]:
df2.shape

(3740, 19)

In [243]:
df2

,airlineEnglish,airlineKorean,airport,airportCode,city,cityCode,flightPurpose,internationalEddate,internationalFri,internationalIoType,internationalMon,internationalNum,internationalSat,internationalStdate,internationalSun,internationalThu,internationalTime,internationalTue,internationalWed
0,T'WAY AIR CO.LTD,티웨이항공,무안,MWX,키타큐슈,KKJ,여객기,2019-03-11T00:00:00+09:00,Y,IN,Y,TW232,Y,2018-11-03T00:00:00+09:00,Y,Y,0015,Y,Y
1,EASTAR JET,이스타항공,청주,CJJ,타이페이/타이베이,TPE,여객기,2019-03-30T00:00:00+09:00,Y,IN,Y,ZE782,N,2018-10-28T00:00:00+09:00,N,N,0510,N,N
2,JEJU AIR,제주항공,대구,TAE,타이페이/타이베이,TPE,여객기,2019-03-31T00:00:00+09:00,Y,IN,Y,7C2686,Y,2018-12-23T00:00:00+09:00,Y,Y,0510,Y,Y
3,T'WAY AIR CO.LTD,티웨이항공,대구,TAE,타이페이/타이베이,TPE,여객기,2019-03-30T00:00:00+09:00,Y,IN,Y,TW664,Y,2018-10-28T00:00:00+09:00,Y,Y,0515,Y,Y
4,JEJU AIR,제주항공,청주,CJJ,타이페이/타이베이,TPE,여객기,2019-03-31T00:00:00+09:00,Y,IN,Y,7C2678,N,2018-12-23T00:00:00+09:00,Y,Y,0520,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329562,SKY ANGKOR AIRLINES,스카이앙코르항공,부산/김해,PUS,씨앰립/시엠립,REP,여객기,2020-03-02T00:00:00+09:00,N,OUT,Y,ZA246,N,2019-12-30T00:00:00+09:00,N,N,0800,N,N
329666,FLY GANGWON,플라이강원,양양,YNY,타이페이/타이베이,TPE,여객기,2020-02-28T00:00:00+09:00,N,OUT,Y,4V201,N,2019-12-30T00:00:00+09:00,N,N,0940,N,N
329964,FLY GANGWON,플라이강원,양양,YNY,타이페이/타이베이,TPE,여객기,2020-02-28T00:00:00+09:00,N,IN,Y,4V202,N,2019-12-30T00:00:00+09:00,N,N,1615,N,N
330700,FLY GANGWON,플라이강원,양양,YNY,타이페이/타이베이,TPE,여객기,2020-02-28T00:00:00+09:00,N,OUT,N,4V201,N,2019-12-31T00:00:00+09:00,N,N,1005,Y,N


In [244]:
df2['internationalStdate'] = df2['internationalStdate'].apply(lambda x: x[:10])
df2['internationalEddate'] = df2['internationalEddate'].apply(lambda x: x[:10])

In [245]:
df2['year'] =df2['internationalStdate'].apply(lambda x : x[:4])

In [246]:
df2['year'].value_counts()

2019    2869
2018     871
Name: year, dtype: int64

## Database 연결 및 Create, Insert 

GRANT ALL PRIVILEGES ON *.* TO 'root'@'%' IDENTIFIED BY '12345'

In [266]:
try:
    con = pymysql.connect(host='3.35.16.111', user='root', port=3306, password='1234', db='sqoop', charset='utf8')
    cur = con.cursor()  #그나마 이상한거라고 있다면 host 주소 인데 namenode 퍼블릭 ip라서 문제 없지 않나... 궁금좌 나와야하는데.. why...
except Exception as e: #주인만 안되는 더러운 세상 넹 ㅠㅠ 아 근데 지금 저거는 제 ip맞지않나요
    print (e)   

In [262]:
for name in df2.keys():
    print("`{}`".format(name))

`airlineEnglish`
`airlineKorean`
`airport`
`airportCode`
`city`
`cityCode`
`flightPurpose`
`internationalEddate`
`internationalFri`
`internationalIoType`
`internationalMon`
`internationalNum`
`internationalSat`
`internationalStdate`
`internationalSun`
`internationalThu`
`internationalTime`
`internationalTue`
`internationalWed`
`year`


In [264]:
create_table = """
    CREATE TABLE airport_info (
       `airlineEnglish`       VARCHAR(30),
       `airlineKorean`        VARCHAR(20),
        `airport`             VARCHAR(20),
        `airportCode`         VARCHAR(20),
        `city`                VARCHAR(20),
        `cityCode`            VARCHAR(20),
        `flightPurpose`       VARCHAR(20),
        `internationalEddate` DATETIME,
        `internationalFri`    VARCHAR(1),
        `internationalIoType` VARCHAR(3),
        `internationalMon`    VARCHAR(1),
        `internationalNum`    VARCHAR(10),  
        `internationalSat`    VARCHAR(1),
        `internationalStdate` DATETIME,
        `internationalSun`    VARCHAR(1),
        `internationalThu`    VARCHAR(1),
        `internationalTime`   TIME,
        `internationalTue`    VARCHAR(1),
        `internationalWed`    VARCHAR(1),
        `year`                YEAR,
    ) COLLATE='utf8mb4_general_ci' 
"""# utf8 > mv4 bgr2rgb

In [265]:
print(create_table)


    CREATE TABLE airport_info (
       `airlineEnglish`       VARCHAR(30),
       `airlineKorean`        VARCHAR(20),
        `airport`             VARCHAR(20),
        `airportCode`         VARCHAR(20),
        `city`                VARCHAR(20),
        `cityCode`            VARCHAR(20),
        `flightPurpose`       VARCHAR(20),
        `internationalEddate` DATETIME,
        `internationalFri`    VARCHAR(1),
        `internationalIoType` VARCHAR(3),
        `internationalMon`    VARCHAR(1),
        `internationalNum`    VARCHAR(10),  
        `internationalSat`    VARCHAR(1),
        `internationalStdate` DATETIME,
        `internationalSun`    VARCHAR(1),
        `internationalThu`    VARCHAR(1),
        `internationalTime`   TIME,
        `internationalTue`    VARCHAR(1),
        `internationalWed`    VARCHAR(1),
        `year`                YEAR,
    ) COLLATE='utf8mb4_general_ci' 



In [268]:
cur.execute(create_table)

ProgrammingError: (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near ') COLLATE='utf8mb4_general_ci'' at line 22")

In [259]:
con.commit()

In [58]:
for name in df.keys():
    print("{}".format(name), end = ", ")

In [199]:
sql = """INSERT INTO airport_info (airlineEnglish, airlineKorean, airport, airportCode, city,
cityCode, flightPurpose, internationalEddate, internationalFri, internationalIoType, internationalMon,
internationalNum , internationalSat, internationalStdate, internationalSun, internationalThu,
internationalTime, internationalTue ,internationalWed ,year) VALUES
         (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

In [ ]:
airlineEnglish, airlineKorean, airport, airportCode, city, cityCode, flightPurpose, internationalEddate, internationalFri, internationalIoType, internationalMon, internationalNum , internationalSat, internationalStdate, internationalSun, internationalThu, internationalTime, internationalTue ,internationalWed ,year

In [60]:
for row in df2.iterrows():
    try:
        cur.execute(sql, row[1].tolist())
    except Exception as e:
        print(e)
        break
        
con.commit()